# Homework 2

## Question 1

In [2]:
!mlflow --version

mlflow, version 2.22.1


## Question 2

In [8]:
!mkdir data
!wget -P data https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-01.parquet
!wget -P data https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-02.parquet
!wget -P data https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-03.parquet

mkdir: cannot create directory ‘data’: File exists
--2025-06-07 08:53:07--  https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-01.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 18.239.38.83, 18.239.38.147, 18.239.38.181, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|18.239.38.83|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1427002 (1.4M) [binary/octet-stream]
Saving to: ‘data/green_tripdata_2023-01.parquet’

green_tripdata_2023 100%[===================>]   1.36M  --.-KB/s    in 0.008s  

2025-06-07 08:53:07 (162 MB/s) - ‘data/green_tripdata_2023-01.parquet’ saved [1427002/1427002]

--2025-06-07 08:53:07--  https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-02.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 18.239.38.83, 18.239.38.181, 18.239.38.163, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfro

In [9]:
!python preprocess_data.py --raw_data_path data --dest_path ./output

In [11]:
!ls -1 output | wc -l

4


## Question 3

Run local server with:

`mlflow server --host 127.0.0.1 --port 5000`

In [14]:
! python train.py

2025/06/07 09:29:06 WARNING mlflow.utils.autologging_utils: MLflow sklearn autologging is known to be compatible with 0.24.1 <= scikit-learn <= 1.6.1, but the installed version is 1.7.0. If you encounter errors during autologging, try upgrading / downgrading scikit-learn to a compatible version, or try upgrading MLflow.
2025/06/07 09:29:06 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2025/06/07 09:29:06 INFO mlflow.tracking.fluent: Experiment with name 'Question 3' does not exist. Creating a new experiment.
2025/06/07 09:29:25 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
🏃 View run bright-asp-972 at: http://localhost:5000/#/experiments/376998915187664384/runs/8e023b7acbf14736a91cc32a623ca94e
🧪 View experiment at: http://localhost:5000/#/experiments/376998915187664384


In [18]:
import mlflow

# Replace with your run ID
run_id = "8e023b7acbf14736a91cc32a623ca94e"

# Get the run object
run = mlflow.get_run(run_id)

# Access a specific parameter
param_name = "min_samples_split"
param_value = run.data.params.get(param_name)

print(f"Value of parameter '{param_name}': {param_value}")

Value of parameter 'min_samples_split': 2


## Question 4

Start mlflow with SQL backend and artifacts storage: 

`mlflow server --backend-store-uri sqlite:///mlflow.db --default-artifact-root ./mlruns_artifacts --host 127.0.0.1 --port 5000`

## Question 5

In [2]:
!python hpo.py

/workspaces/mlops_zoomcamp/02-experiment-tracking/.venv/lib/python3.12/site-packages/hyperopt/atpe.py:19: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
2025/06/07 12:23:31 INFO mlflow.tracking.fluent: Experiment with name 'random-forest-hyperopt' does not exist. Creating a new experiment.
🏃 View run colorful-moose-795 at: http://127.0.0.1:5000/#/experiments/1/runs/f13f07da4295404d853aebb1d8eb41c8

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1                    

🏃 View run rare-kite-547 at: http://127.0.0.1:5000/#/experiments/1/runs/933e227345904066b0ae83f2ee0892e1

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1                    

🏃 View run omniscient-goat-632 at: http://127.0.0.1:5000/#/experiments/1/runs/85c7b93ed633480fb4057cdd74c0e068



In [7]:
import mlflow
from mlflow.tracking import MlflowClient

client = MlflowClient()

experiment_name = "random-forest-hyperopt"
experiment = client.get_experiment_by_name(experiment_name)

if experiment is None:
    raise ValueError(f"Experiment '{experiment_name}' not found.")

experiment_id = experiment.experiment_id

# Just sort by metric; MLflow will automatically exclude runs without this metric
runs = client.search_runs(
    experiment_ids=[experiment_id],
    filter_string="",  # no filter
    order_by=["metrics.`validation-rmse` ASC"],
    max_results=1
)

if not runs:
    raise ValueError("No runs found with 'validation-rmse' metric.")

best_run = runs[0]
best_rmse = best_run.data.metrics["validation-rmse"]

print(f"Best validation RMSE: {best_rmse}")
print(f"Run ID: {best_run.info.run_id}")


Best validation RMSE: 5.335419588556921
Run ID: f0f464a2b57e450c8a4eb31798ba90af


## Question 6

In [15]:
!python register_model.py

2025/06/07 13:02:10 WARNING mlflow.utils.autologging_utils: MLflow sklearn autologging is known to be compatible with 0.24.1 <= scikit-learn <= 1.6.1, but the installed version is 1.7.0. If you encounter errors during autologging, try upgrading / downgrading scikit-learn to a compatible version, or try upgrading MLflow.
2025/06/07 13:02:20 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
🏃 View run amazing-shark-563 at: http://127.0.0.1:5000/#/experiments/2/runs/40abcede2e7442948d0079c0ebf3a98d
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/2
2025/06/07 13:02:30 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
🏃 View run unleashed-eel-128 at: http://127.0.0.1:5000/#/experiments/2/runs/ae6bf083682048d8b603bc257e020506
🧪 View experiment at: http://127.0.0.1:5000/#/